In [67]:
import re
import time
import json
import pymysql
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def kyochon_store():
    KYOCHON_URL = "http://www.kyochon.com/shop/domestic.asp"
    wd = webdriver.Chrome()
    wd.get(KYOCHON_URL)

    result = []

    try:
        # WebDriverWait 객체 생성 (최대 10초까지 기다림)
        wait = WebDriverWait(wd, 10)

        # 1. '서울' 버튼이 클릭 가능해질 때까지 기다린 후 클릭
        # 교촌 사이트는 '서울' 버튼의 JavaScript 함수 `searchShop('sido', '01')`를 직접 호출해 보겠습니다.
        # 또는 아래와 같이 클릭으로 처리해도 됩니다.
        # seoul_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@href=\"javascript:searchShop('sido', '01');\"]")))
        # seoul_button.click()
        # wd.execute_script("searchShop('sido', '01')") # '서울' 지역 검색 실행

        # 2. 검색 결과로 매장 목록이 담긴 'ul' 태그가 나타날 때까지 기다림
        # wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "ul.list")))

        time.sleep(1) # 데이터가 완전히 렌더링될 때까지 아주 잠시 대기

        html = wd.page_source
        soup = BeautifulSoup(html, 'html.parser')

        store_list = soup.select("ul.list > li")

        if not store_list:
            print("❌ 매장 정보를 찾을 수 없습니다.")
            return []

        print("\n4. 매장 정보 추출을 시작합니다...")
        for store in store_list:
            name = store.select_one("strong").get_text(strip=True)
            info_text = store.select_one("em").get_text(strip=True).replace('\n', ' ').replace('\r', ' ')

            # --- 여기가 핵심: 주소와 전화번호 분리 ---
            address = info_text
            phone = "정보 없음"

            # 전화번호 패턴: 0으로 시작하고, 숫자, 하이픈(-), 쉼표(,), 공백으로 구성된 문자열이 끝에 오는 경우
            # 예: 02-123-4567, 02 - 1234 - 5678, 031-111-2222,9999 등
            phone_pattern = re.compile(r'(\s*0\d{1,2}[\s-]*\d{3,4}[\s-]*\d{4}[,\d~]*)?$')

            match = phone_pattern.search(address)
            if match and match.group(1): # 전화번호 패턴이 찾아졌다면
                phone_number_part = match.group(1).strip()
                # 전체 문자열에서 전화번호 부분을 제거하여 주소를 만듦
                address_part = address[:match.start()].strip()

                # 가끔 주소 끝에 ')'가 붙어있지 않은 경우를 대비해 정리
                if address_part.endswith(')'):
                    phone = phone_number_part
                    address = address_part
                else: # '...1층)02-...' 형태가 아닌 '...1층 02-...' 형태일 경우
                    # 마지막 공백을 기준으로 분리 시도
                    last_space_index = info_text.rfind(' ', 0, match.start())
                    if last_space_index != -1:
                        address = info_text[:last_space_index].strip()
                        phone = info_text[last_space_index:].strip()
                    else: # 공백이 없는 예외적인 경우
                        address = address_part
                        phone = phone_number_part
            # 주소와 전화번호 분리 (더 견고한 분리 로직이 필요할 수 있음)
            # address = info_text
            # phone = "전화번호 없음" # 전화번호는 별도로 파싱해야 할 수 있음

            print(f"📍 매장명: {name} | 📍 주소: {address} | 📞 전화번호: {phone}")
            result.append({"store": name, "address": address, "phone": phone})

    except Exception as e:
        print(f"❌ 오류 발생: {e}")

    finally:
        wd.quit()
        return result

In [68]:
# JSON 파일 저장 ---
def save_to_json(data):
    json_file = "kyochon.json"
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"✅ JSON 파일 저장 완료: {json_file}")

# MariaDB(MySQL) 저장 ---
def save_to_mariadb(data):
    db = pymysql.connect(
        host="localhost",
        user="webuser",  # 변경 필요
        password="webuser",  # 변경 필요
        database="webdb"
    )
    cursor = db.cursor()

    # 테이블 생성 (처음 실행 시 필요)
    create_table_query = """
    CREATE TABLE IF NOT EXISTS kyochon_stores (
        id INT AUTO_INCREMENT PRIMARY KEY,
        store VARCHAR(255) NOT NULL,
        address TEXT NOT NULL,
        phone VARCHAR(100)
    )
    """
    cursor.execute(create_table_query)

    # 데이터 저장 쿼리
    insert_query = """
    INSERT INTO kyochon_stores (store, address, phone)
    VALUES (%s, %s, %s)
    """

    try:
        cursor.executemany(insert_query, [(d["store"], d["address"], d["phone"]) for d in data])
        db.commit()
        print(f"✅ {len(data)}개의 매장 데이터가 MariaDB에 저장되었습니다.")
    except Exception as e:
        db.rollback()
        print(f"❌ 데이터 저장 중 오류 발생: {e}")

    cursor.close()
    db.close()

In [69]:
def main():
    print("🚀 교촌 매장 정보 크롤링 시작...")
    store_data = kyochon_store()

    print(f"store_data 확인 : {store_data} ")
    # 파일 기반 저장,
    print("\n💾 JSON 파일 저장 중...")
    save_to_json(store_data)
    # 데이터베이스 저장.
    print("\n📂 MariaDB 저장 중...")
    save_to_mariadb(store_data)

    print("\n🎉 모든 작업 완료!")

if __name__ == '__main__':
    main()

🚀 교촌 매장 정보 크롤링 시작...

4. 매장 정보 추출을 시작합니다...
📍 매장명: 가락2호 | 📍 주소: 서울 송파구 가락동 75-7(서울 송파구 양재대로 62길 35 다우빌딩 1층) | 📞 전화번호: 02 -448-9005,9045
📍 매장명: 가산디지털 | 📍 주소: 서울 금천구 가산동 371-6 가산비지니스센터 2층(서울 금천구 가산디지털1로 165 가산비지니스센터 2층) | 📞 전화번호: 02 -865-0865
📍 매장명: 가양 | 📍 주소: 서울 강서구 등촌동 75-1(서울 강서구 양천로 482 삼부르네상스 109호) | 📞 전화번호: 02 -3665-3433
📍 매장명: 강남역 | 📍 주소: 서울 강남구 역삼동 619-28(서울 강남구 강남대로 98길 11) | 📞 전화번호: 02 -596-1991
📍 매장명: 강서구청 | 📍 주소: 서울 강서구 화곡동 986-16(서울 강서구 화곡로 277) | 📞 전화번호: 02 -2699-3999
📍 매장명: 강일역 | 📍 주소: 서울시 강동구 강일동 679-3(서울 강동구 아리수로 419 강일리슈빌에스 1층 101,102호) | 📞 전화번호: 02 -442-1994
📍 매장명: 개봉1호 | 📍 주소: 서울 구로구 개봉동 403-159(서울 구로구 개봉로 30-1) | 📞 전화번호: 02 -2066-9244~5
📍 매장명: 개포1호 | 📍 주소: 서울 강남구 개포동 1229-18(서울 강남구 논현로16길 5) | 📞 전화번호: 02 -573-5699,5499
📍 매장명: 개포2호 | 📍 주소: 서울 강남구 개포동 158-4(서울 강남구 선릉로6길 12(개포동)) | 📞 전화번호: 02 -5779667
📍 매장명: 거여가락1호 | 📍 주소: 서울 송파구 가락동 180-9(서울 송파구 문정로 143) | 📞 전화번호: 02 -406-9282
📍 매장명: 건대 | 📍 주소: 서울 광진구 화양동 9-18(서울 광진구 아차산로 31길 13) | 📞 전화번호: 02 -462-1991
📍 매장명: 경희대 | 📍 주